# Introduction

To begin with, these are described in the order they are handled in the original program. They will be combined/expanded/rearranged as the project evolves.

Note: All equations are from the previous code, unless specified otherwise.

In [ ]:
import numpy as np
from scipy import constants as const
import math

    global e_amu
        
    e_amu = 931.5 # rest energy of 1 AMU in MeV
    
    global c
        
    c = float(const.c) # speed of light
    
    global pi
    
    pi = float(const.pi) #pi
    
    global degtorad
    
    degtorad = -0.017453 # -1 degree in radians
    
    

# User Input

Information regarding the mass, charge states, and starting linac section needs to be gathered from the user.

Additionally, this part will need to have a part asking if the user wants to enter more than one beam.

Should this be a beam-by-beam entry or use a GUI? It needs to go into the control system so I probably want to keep it as simple as possible. Then again, simple GUI elements should work in the control system. Mass/Energy/charge state text input boxes, Start With PII/Booster/ATLAS checkboxes and if PII/Booster then check the boxes after it. If entering PII then use 30.5 keV/u RFQ acceptance energy (or maybe RFQ exit energy if we're going straight into the PII resonators?).

Also: scale factor (default to 1)

# Database Import--not doing the actual import part anymore, Controls Group will do this.

Input resonator field data from databases, probably using string search with mySQL or similar package. Assign these db values to variables (done as used during each subroutine in the old code but not reason this can't all happen here as the db files are not changed by the utility).

Calculate $El_i$ for each resonator, and lorentz $\beta$, $\gamma$, and energy.

**Note:** the $\gamma$ in this program is actually, somewhat confusingly, $\frac{1}{\gamma}$ if $\gamma$ is the Lorentz $\gamma$. The $\beta$ is confirmed to be the Lorentzian $\beta$, in terms of the relativistic ion rest energy and kinetic energy instead of velocity. The math works out.

$El_i = (Amplitude\_Calibrate\_Intercept + Amplitude\_Calibrate\_Slope\ast Amplitude\_After\_Scan)\ast Amplitude\_Scale\_Factor$

\begin{equation}
    \beta(1) = \sqrt{1-(1 + \frac{Total\_Energy}{e\_amu\ast Mass})^{-2}}
\end{equation}

\begin{equation}
    \gamma 0 = \sqrt{1 - \beta (I)^2}
\end{equation}

\begin{equation}
    Ener0 = Mass\ast e\_amu\ast (\frac{1}{\gamma 0} - 1)
\end{equation}

Variables assigned elsewhere that could be assigned here:

Frequency (takes the frequency from the file and multiplies by $10e^{-6}$ to convert to MHz.)

Cavity lengths

And of course error handling for RES_VDB database retrieval and printing.

**Then it calls the Resonator Position Subroutine**



# Resonator Position Subroutine

Defining the resonator position for each resonator. The old code did this with a "do while" loop, starting with Totl = 0.0, I = 1. It checked the EAX number designation (21, 31-33, 41-44) and assigned "unit = 2", the appropriate .dat file, and status = old to all of them. I think the unit # and status = old may be unnecessary outside of blocking out the logic cell in Fortran.

\begin{equation}
    Totl = Totl+(Z2-Z1)\ast Msh
\end{equation}

So Z2-Z1 is the distance step size in the Eax file, then Msh is the number of steps in that file. (Note: A similar treatment is given by the variable "Mesh" for resonators given "unit = 3" in the Resonator_Calculation subroutine.) Then it assigns Zelm(I) = Totl, so it's assigning the position to a unique variable for each resonator.

Then I is incremented by 1, and another iteration of the loop occurs.

Done 06/2020 *Task: look through the databases and confirm what Msh is*

Question: It doesn't seem like we care about drift spaces, we just assume an uninterrupted string of unit cells. But the phase ellipse shears in phase spaces so will that make the phases inaccurate between linac sections? Or does that even matter because all we care about here is energy?

# Headings

Headings listed for the table should be: Beam index number (since we are doing multiple beams), Mass, Entrance Energy, total resonators, amplitude after scan, scale factor used. It would also be nice to add a section header for each linac section (PII/Booster/ATLAS) and have a little break between each section but that might be too much for right now.

# Calculate Energy Gains Subroutine

The subroutine begins by assigning charge states to the appropriate accelerating sections (will probably do earlier).

Deactivated resonators are skipped, and the Lorentzian $\beta$ calculated for ions exiting that resonator will be fed to the next resonator.

Then the phase offset is calculated: 

\begin{equation}
    Poff = degtorad\ast Resonator\_Data\_Table(Record).Default\_Acceleration\_Phase
\end{equation}

**Then it calls the Resonator_Calculation subroutine, which will be addressed in another section.**

Then "Zinner" values are the cavity length (the accelerating distance) assigned to resonators corresponding to the EAX file designator. Type 21 $\rightarrow$ Zinner = 0.390, 31 $\rightarrow$ 0.208, 32 $\rightarrow$ 0.355, 41 $\rightarrow$ 0.102, 42 $\rightarrow$ 0.165, 43 $\rightarrow$ 0.254.

## Aside:

Note: equations in this "aside" are from theoretical text, not code.

Based on the energy gain equation 

\begin{equation}
    \Delta W = qE_0l_iT(k)I_0(k_rr)\cos{\phi}
\end{equation}

where $q$ is the ion charge, $E_0$ is the average longitudinal field on axis in a cell, $l_i$ is the cell length, T(k) is the transit time factor on axis in a cell, and $I_0$ is the modified Bessel function of order zero.

$E_0l_i$ appears to be calculated in the Database Import section for each resonator, so for the moment I'm assuming that $El = E_0l_i$. (If we assume that $E_0$ is zero in each drift space, then $\Delta W = 0$ in the drift space also.) The transit time factor, which accounts for the time variation of the field experienced by the moving particle, can be expressed by

\begin{equation}
    T = \frac{\sin{(\pi L/\beta\lambda)}}{\pi L/\beta\lambda} = \frac{\sin{(\omega L/2v)}}{\omega L/2v}
\end{equation}

(which comes from)

\begin{equation}
    T(k) = \frac{\int_{0}^{L}E_z(z)e^{-j\frac{\omega z}{v_p}}dz}{\int_{0}^{L}E(z)dz}
\end{equation}

where the top expression is the time-varying field and the bottom expression is the maximum field. This is multiplied by the modified Bessel function of order zero $I_0(k_rr)$ where

\begin{equation}
    k_r = \frac{2\pi}{\gamma\beta\lambda}
\end{equation}

The ttf is later calculated backwards from the Umatch variable, which gives further evidence that this is the case.

*Task: opportunity to check against old code by independently calculating ttf and comparing*

## Continuing on:

The energy is calculated, which resembles the energy gain equation mentioned previously:

\begin{equation}
    U_{match} = Charge\ast El(I)\ast Zinner\ast \cos{P_{off}}
\end{equation}

followed by the descriptively named "term" variable, which is really just the Lorentz $\gamma$ of one resonator minus the $\gamma$ of the previous resonator:

\begin{equation}
    Term = \frac{1}{(\sqrt{1-\beta_{Next\_Pointer})^2}} - \frac{1}{(\sqrt{1-\beta_{Current\_Pointer})^2}}
\end{equation}

Next the ttf is calculated from the $Umatch$ variable:

\begin{equation}
    Ttf(Record) = e\_amu\ast Mass\ast \frac{Term}{Umatch}
\end{equation}

Then "I" and "Record" are incremented by 1, and we reiterate the loop.

*Task: figure out more precisely how Current_Pointer and Next_Pointer work*

# Resonator Calculation Subroutine

**NOTE: Omc is the group velocity of the RF field wave through the cavity, and Uzero is the resonator exit energy that gets printed in the final results**

Series of elif statements checking the Eax_File value and opening the appropriate Eax(#).dat file and assigning unit = 3 and status = "OLD". Again the unit and status appear to be for identifying the logic cell.

Next it reads "Mesh" and "Freq" from the appropriate eax.dat files.

Next it reads values from the .dat files and assigns them to Z0(Count) and Ez0(Count) *while* the "Count" index is less than or equal to "Mesh" (which, as in the Resonator_Position subroutine, seems to be related to the number of resonators).

Then we assign some variables:

\begin{equation}
    Om = 2\ast\pi\ast Freq
\end{equation}

\begin{equation}
    Omc = \frac{Om}{c}
\end{equation}

\begin{equation}
    Qdm0 = \frac{Charge\ast 9.580838e^7}{Mass\ast c}
\end{equation}

\begin{equation}
    Qdm0c = \frac{Qdm0}{c}
\end{equation}

**Then it calls the Phind subroutine**

Based on the phase matching done in the Phind subroutine, phase variables are used to define the following initial conditions, which will then be recalculated for exit conditions, iteratively redefined, and fed into the next resonator for its starting conditions:

\begin{equation}
    Phitmp = Phi00 + Poff
\end{equation}

\begin{equation}
    Btmp = B0
\end{equation}

\begin{equation}
    Dz = Z0(2) - Z0(1)
\end{equation}

\begin{equation}
    Timeout = Timein
\end{equation}

\begin{equation}
    J = 1
\end{equation}

where J is the counting index for the "do while J $\leq$ Mesh" loop. That loop goes like this:

\begin{equation}
    Phi0(J) = Phitmp
\end{equation}

\begin{equation}
    Beta0(J) = Btmp
\end{equation}

\begin{equation}
    Eztmp = El\ast Ez0(J)\ast \cos{Phitmp}
\end{equation}

\begin{equation}
    Phitmp = Phitmp + \frac{Omc\ast Dz}{Btmp}
\end{equation}

\begin{equation}
    Btmp = Btmp + \frac{Eztmp\ast Qdm0c\ast Dz}{Btmp}
\end{equation}

\begin{equation}
    Timeout = Timeout + \frac{Dz}{Btmp\ast c}
\end{equation}

And then the index is incremented: $J = J + 1$ and the "do while" loop ends.

Finally the resonator exit energy is calculated:

\begin{equation}
    Uzero = e\_amu\ast Mass\ast (\frac{1}{\sqrt{1-Beta0(Mesh)^2}}-1)
\end{equation}

which can be simplified in the future to:

\begin{equation}
    Uzero = e\_amu\ast Mass\ast (\frac{1}{\gamma (Mesh)} -1)
\end{equation}

And as far as I can tell, the following are calculated but never used anywhere:

\begin{equation}
    Bf = Beta0(Mesh)
\end{equation}

\begin{equation}
    Term1 = Mass\ast B0\ast (e\_amu\ast 1000)
\end{equation}

\begin{equation}
    Term2 = Term1\ast \frac{Bf}{B0}
\end{equation}

Note: Btmp gets passed to B0, then Beta0(J) is assigned to Btmp, Btmp is changed before incrementing J at the end of the do loop, and then Beta0(Mesh) gets assigned to Bf (probably stands for $\beta$ Final). **This needs to be more clearly organized, and also clearly named (e.g. Term, Term1, Term2).**

*Task: get an eax.dat file from Chris and look through it* (Done--this is 200 lines with distance values (-d/2 to +d/2 with 0 at resonator center) and corresponding field values)

*Task: confirm suspicion of what "Mesh" variable is by looking at database files*

# Phind Subroutine

**Note on the code: In Resonator_Calculation, Phind is called with certain arguments, but when Phind is actually defined, the arguments have slightly different names but each series of arguments corresponds with the other. This means that the same variables are called different things in the code. This can be really confusing so I am placing this here. Phind(El, Ez0, Z0, Mesh, B0, Freq, Phi00, Ug00, Qdm0c) = Phind(El, Eax, Zax, N, Beta, Freq, Phi, Vgain, Qdm) and their arguments correspond in order (i.e. Ez0 = Eax and both are the E-field level as read from the 200 values in the Eax.dat file; Z0 = Zax and both are the position as read from the 200 positional values in the Eax.dat file; Mesh = N; B0 = Beta, etc.)**

This subroutine attempts to maximize the phase angle by calculating $\beta$ for different phase angles in one degree increments. If the new $\beta$ is bigger than the old $\beta$, the loop iterates again, and if not then the last degree was the maximum phase angle. It alleges that it does a parabolic interpolation but I don't see how this is actually the case; it just looks like a weighted average to me at the moment.

It starts by defining variables, including the logical statement $newmax$, before launching into a series of "do" (for) loops. Logic blocks are numbered by fives and call each other as below in color text.

Variables defined:

\begin{equation}
    Newmax = FALSE
\end{equation}

\begin{equation}
    Om = 2\ast \pi\ast Freq
\end{equation}

\begin{equation}
    Phitmp = \frac{-\pi}{360}
\end{equation}

(which sets the starting phase)

\begin{equation}
    Dphi = \frac{\pi}{180}
\end{equation}

(which sets the $1^{\circ}$ step size)

\begin{equation}
    Dz = \frac{|Zax(N) - Zax(1)|}{N - 1}
\end{equation}

(note: make sure $N-1$ above is a float not an integer.)

\begin{equation}
    Pre = El\ast Qdm\ast Dz
\end{equation}

\begin{equation}
    Bmax = 0
\end{equation}

(this sets $\beta_{max}$ to zero for the first iteration so the next point will automatically produce a Newmax = TRUE for nonzero acceleration)

\begin{equation}
    Bfor = Beta
\end{equation}

\begin{equation}
    T = 0
\end{equation}

<hr>

We start by calling the first block with "do $\color{blue}{\text{5}}$ I=1,N" so we have our index set up, and do the following calculations for each iteration of the index:

*Comment from code: "Calculate one point outside loop to initialize parabolic fit into correctly."*

do $\color{blue}{\text{5}}$ continue

\begin{equation}
    Bfor = Bfor + \frac{Pre\ast Eax(I)\ast \cos{(Om\ast T + Phitmp)}}{Bfor}
\end{equation}

\begin{equation}
    T = T + \frac{Dz}{Bfor\ast c}
\end{equation}

(which increments time elapsed in a similar way that the Resonator_Position subroutine increments the distance of the positions down the beamline.)

$\color{blue}{\text{5}}$ continue

do $\color{orange}{\text{20}}$ J=1,50

\begin{equation}
    Btmp = Beta
\end{equation}

\begin{equation}
    T = 0
\end{equation}

\begin{equation}
    Phitmp = Phitmp + Dphi
\end{equation}

do $\color{teal}{\text{10}}$ I=1,N

\begin{equation}
    Btmp = Btmp + \frac{Pre\ast Eax(I)\ast \cos{(Om\ast T +Phitmp)}}{Btmp}
\end{equation}

\begin{equation}
    T = T + \frac{Dz}{Bfor\ast c}
\end{equation}

<hr>

$\color{teal}{\text{10}}$ continue

if Newmax = TRUE then

\begin{equation}
    Bmor = Btmp
\end{equation}

Comment regarding above: "Save velocity at +Dphi from present max."

Newmax = False

endif

if Btmp $\>$ Bmax then

\begin{equation}
    Bmax = Btmp
\end{equation}

\begin{equation}
    Bles = Bfor
\end{equation}

\begin{equation}
    Newmax = TRUE
\end{equation}

\begin{equation}
    Phi = Phitmp
\end{equation}

endif

\begin{equation}
    Bfor = Btmp
\end{equation}

Comment regarding above: "Hold last beta value to associate with Bmax"

<hr>

Comment: "If the last calculated phase had the largest energy gain, then it is necessary to calculate the next point in the phase loop."

$\color{orange}{\text{20}}$ continue

If Newmax = TRUE then

\begin{equation}
    Btmp = Beta
\end{equation}

\begin{equation}
    T = 0
\end{equation}

\begin{equation}
    Phitmp = Phitmp + Dphi
\end{equation}

do $\color{pink}{\text{25}}$ I=1,N

\begin{equation}
    Btmp = Btmp + \frac{Pre\ast Eax(I)\ast \cos{(Om\ast T + Phitmp)}}{Btmp}
\end{equation}

\begin{equation}
    T = T + \frac{Dz}{Btmp\ast c}
\end{equation}

$\color{pink}{\text{25}}$ continue

endif

<hr>

Comment: "Now do a parabolic interpolation on the three points bracketing the maximum to do (sic) determine the maximum phase angle."

\begin{equation}
    Phiint = -Dphi\ast (0.5 + \frac{Bmor^2 - Bmax^2}{Bmor^2-2\ast Bmax^2 + Bles^2})
\end{equation}

\begin{equation}
    Phi = Phiint + Phi + Dphi
\end{equation}

\begin{equation}
    Vgain = \frac{c^2\ast (Bmax^2 - Beta^2)}{2\ast Qdm}
\end{equation}

And now we have our phase angle and gained energy.

# Other Subroutines

The remaining subroutines are:

The List_Settings subroutine, which looks at the database values and print "Resonator Deactivate" or "Resonator Data Table, Default Acceleration Phase, Resfreq(I), Axis, El(I), Eax_File(I), and Resonator ID for each resonator, along with Mass, Initial Energy (Ener0), Beta(1) as MeV Beta, and PII/Booster/ATLAS charge states.

The Display_Results subroutine, which prints (to the command window) the "wall of text" that we usually get on the printer.

The Options_Menu subroutine which allows the user to select: Change beam type, calculate energy gains (which will likely be a redundant option in the future because we want the user to have to prompt the program as few times as possible to get it to do what it's purposed to do), change device settings, list present hardware settings, display results, print results, restart, and quit.



In [2]:
pip install plotly==4.8.2

Note: you may need to restart the kernel to use updated packages.
